# Geodatenanalyse 2


## Termin Big Data 5 - Modul 2

## *Earth Engine*: Analyse und Export

Ca. 20-30 Minuten

## Inhalt

- Datenanalyse in *Google Earth Engine*
- *ee.Features* inspizieren
- Datensätze exportieren
- Zeitreihen exportieren

Beispiele aus diesem [Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

In [1]:
import geemap
import geemap.colormaps as cm
import datetime as dt
import pandas as pd

In [2]:
import ee
# initialize the connection to the server
ee.Initialize()

## Datenanalyse in *Google EE*

- *EE* kann als Cloud-Computer zur Berechnung von Datensätzen verwendet werden

- Als Beispiel verwenden wir das [digitale Höhenmodell von NASA/CGIAR](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4)

In [3]:
# das Höhenmodell
srtm = ee.Image('CGIAR/SRTM90_V4')

### Beispiel: Wert für einen Punkt bestimmen

Oftmals wollen wir die Werte eines Rasters für einen bestimmten Punkt bestimmen:

In [4]:
# Kalrsuhe ...
point = ee.Geometry.Point(8.4, 49.0)

# extract the image value
data = srtm.select("elevation") \
    .reduceRegion(ee.Reducer.first(), point, 10) \
    .get("elevation")

data.getInfo()

118

### Beispiel: Gefälle berechnen

Hier wird die Funktion *slope* verwendet, welche bereits eingebaut ist

In [5]:
# Berechnung des Gefälles
slope = ee.Terrain.slope(srtm)

# Ergebnis anzeigen ...
globe = geemap.Map()
globe.setCenter(6.2, 46.210, 8)
globe.addLayer(slope, {'min': 0, 'max': 60, 'palette': cm.palettes.jet}, 'slope', opacity=0.7)
globe

Map(center=[46.21, 6.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Trick. Eine Region auf der Karte auswählen:

In [8]:
select = globe.draw_last_feature.geometry().getInfo()
select

{'type': 'Polygon',
 'coordinates': [[[5.680486, 45.334417],
   [6.487642, 45.334417],
   [6.487642, 45.684725],
   [5.680486, 45.684725],
   [5.680486, 45.334417]]]}

### Beispiel: Statistik des Gefälles

In [9]:
# Median des Gefälles berechnen
stats = slope.reduceRegion(
    reducer = ee.Reducer.median(),
    geometry = select,
    scale = 30,
    maxPixels = 1e9)

stats.getInfo()

{'slope': 18.250664370663035}

Eine Kombination:

In [10]:
# a combination of reducers
red_comb = ee.Reducer.mean().combine(
  reducer2 = ee.Reducer.stdDev(),
  sharedInputs = True
).combine(
  reducer2 = ee.Reducer.min(),
  sharedInputs = True
).combine(
  reducer2 = ee.Reducer.max(),
  sharedInputs = True
)

# Min, max and standard deviation
stats = slope.reduceRegion(
    reducer = red_comb,
    geometry = select,
    scale = 30,
    maxPixels = 1e9)

stats.getInfo()

{'slope_max': 74.73391723632812,
 'slope_mean': 18.665135591517714,
 'slope_min': 0,
 'slope_stdDev': 12.510407775860152}

## Datensätze exportieren

Eine generelle Lösung für den Export von Datensätzen ist es, diese gezielt zum Download vorzubereiten:

In [11]:
# Get a download URL for an image
image = ee.Image('srtm90_v4')

path = image.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': [[-120, 35], [-119, 35], [-119, 34], [-120, 34]]
})

print(path)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6372828bf91c925d155cb0ea8c792d61-3d3d048d8592e19318a9d2715c6e16a8:getPixels


## *ee.FeatureCollections* inspizieren

Zuerst laden wir die Ländergrenzen aus einem *ee.FeatureCollection* des [US Department of State, Office of Geography](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017):

In [12]:
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

Schauen wir uns die Metadaten eines Elements der Kollektion an:

In [13]:
countries.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[15.799896950556025, 7.442973734412789],
    [15.805822514459608, 7.44729739764575],
    [15.820636359112441, 7.4477452116941585],
    [15.822772337809184, 7.4466428006736205],
    [15.830765007792202, 7.452568395501756],
    [15.833727722314663, 7.451982675764594],
    [15.837551791116674, 7.454911057681969],
    [15.841555568388516, 7.455523671252298],
    [15.871210387880351, 7.460061474298881],
    [15.877756073520416, 7.464574548261912],
    [15.8907096764652, 7.466348725021928],
    [15.895360549347515, 7.4690703805707885],
    [15.90755617246162, 7.4722570979530625],
    [15.914928669232438, 7.4766668272593355],
    [15.916616783923898, 7.480714753028353],
    [15.922783482791532, 7.483057477390312],
    [15.928709002279916, 7.48267848797983],
    [15.932980941135774, 7.484935063540838],
    [15.943833049585004, 7.484332134135637],
    [15.956028664871996, 7.485641310730861],
    [15.966811805433828, 7.48872

Eine Liste der Länder erstellen:

In [14]:
contry_list = countries.aggregate_array('country_na')
contry_list.getInfo()[:20]

['Chad',
 'Malawi',
 'Zambia',
 'Zimbabwe',
 'Botswana',
 'Namibia',
 'Angola',
 'Burundi',
 'Rwanda',
 'South Africa',
 'Lesotho',
 'Swaziland',
 'Mayotte',
 'Niger',
 'Sudan',
 'Libya',
 'Koualou Area',
 'Tunisia',
 'Egypt',
 'Bir Tawil']

Wo ist Deutschland? Finden wir das Polygon:

In [15]:
germany = countries.filter(ee.Filter.eq('country_na', 'Germany'))
germany.getInfo()

{'type': 'FeatureCollection',
 'columns': {'abbreviati': 'String',
  'country_co': 'String',
  'country_na': 'String',
  'system:index': 'String',
  'wld_rgn': 'String'},
 'id': 'USDOS/LSIB_SIMPLE/2017',
 'version': 1566850254549599,
 'properties': {'date_range': [1490832000000, 1490832000000],
  'period': 0,
  'thumb': 'https://mw1.google.com/ges/dd/images/LSIB_thumb.png',
  'description': '<p>The United States Office of the Geographer provides\nthe Large Scale International Boundary (LSIB) dataset. The detailed\nversion (2013) is derived from two other datasets: a LSIB line\nvector file and the World Vector Shorelines (WVS) from the National\nGeospatial-Intelligence Agency (NGA). The interior boundaries\nreflect U.S. government policies on boundaries, boundary disputes,\nand sovereignty. The exterior boundaries are derived from the\nWVS; however, the WVS coastline data is outdated and generally\nshifted from between several hundred meters to over a kilometer.\nEach feature is the pol

Holen wir uns die WGS84 Koordinaten des Zentrums als Information:

In [16]:
germany.geometry().centroid().getInfo()['coordinates']

[10.372575736879764, 51.04952573474018]

Schauen wir uns das Polygon in einer Karte an:

In [17]:
globe = geemap.Map(location=[49.014, 8.405], zoom_start=4)

vis_params = {'color': "red", 'width': 0.5}
globe.addLayer(germany, vis_params, name='Germany')
globe.centerObject(germany)

globe

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

## *ee.Feature* exportieren

Das Polygon als Shapefile exportieren:

In [18]:
geemap.ee_export_vector(germany, 'data/germany.shp')

Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data\germany.shp


## *ee.Image* Regionen exportieren

In [19]:
globe = geemap.Map(location=[49.014, 8.405], zoom=10)

image = ee.Image('USGS/SRTMGL1_003')

vis_params = {'min': 100, 'max': 800, 'palette': cm.palettes.dem}
globe.addLayer(image, vis_params, 'SRTM DEM', True, 0.9)

KA = ee.Geometry.Point([8.407, 49.015])
globe.addLayer(KA)

globe

Map(center=[49.014, 8.405], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

### Regional begrenztes Gebiet exportieren

In [20]:
roi = ee.Geometry.Rectangle([8.1, 48.9, 8.7, 49.1])
image_clipped = image.clip(roi)

geemap.ee_export_image(image_clipped, "data/DEM_Karlsruhe.tif", region=roi, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data


### Gebiet in der Karte auswählen

In [22]:
polygon = globe.draw_last_feature.geometry()

geemap.ee_export_image(image, "data/DEM_selected.tif", region=polygon, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data


## *ee.ImageCollections* inspizieren

Hier verwenden wir stündliche Klimareanalysedaten [ERA5 des European Centre for Medium-Range Weather Forecasts](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY). Zuerst schauen wir uns die Dateneigenschaften an:

In [23]:
collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
collection.first().bandNames().getInfo()

['dewpoint_temperature_2m',
 'temperature_2m',
 'skin_temperature',
 'soil_temperature_level_1',
 'soil_temperature_level_2',
 'soil_temperature_level_3',
 'soil_temperature_level_4',
 'lake_bottom_temperature',
 'lake_ice_depth',
 'lake_ice_temperature',
 'lake_mix_layer_depth',
 'lake_mix_layer_temperature',
 'lake_shape_factor',
 'lake_total_layer_temperature',
 'snow_albedo',
 'snow_cover',
 'snow_density',
 'snow_depth',
 'snow_depth_water_equivalent',
 'snowfall',
 'snowmelt',
 'temperature_of_snow_layer',
 'skin_reservoir_content',
 'volumetric_soil_water_layer_1',
 'volumetric_soil_water_layer_2',
 'volumetric_soil_water_layer_3',
 'volumetric_soil_water_layer_4',
 'forecast_albedo',
 'surface_latent_heat_flux',
 'surface_net_solar_radiation',
 'surface_net_thermal_radiation',
 'surface_sensible_heat_flux',
 'surface_solar_radiation_downwards',
 'surface_thermal_radiation_downwards',
 'evaporation_from_bare_soil',
 'evaporation_from_open_water_surfaces_excluding_oceans',
 'evap

Was für Eigenschaften hat die Zeitreihe:

In [24]:
dates = collection.reduceColumns(ee.Reducer.toList(), ["system:time_start"]).get('list')
datelist = pd.to_datetime(dates.getInfo(), unit='ms')
datelist

DatetimeIndex(['1950-01-01 01:00:00', '1950-01-01 02:00:00',
               '1950-01-01 03:00:00', '1950-01-01 04:00:00',
               '1950-01-01 05:00:00', '1950-01-01 06:00:00',
               '1950-01-01 07:00:00', '1950-01-01 08:00:00',
               '1950-01-01 09:00:00', '1950-01-01 10:00:00',
               ...
               '2023-06-22 10:00:00', '2023-06-22 11:00:00',
               '2023-06-22 12:00:00', '2023-06-22 13:00:00',
               '2023-06-22 14:00:00', '2023-06-22 15:00:00',
               '2023-06-22 16:00:00', '2023-06-22 17:00:00',
               '2023-06-22 18:00:00', '2023-06-22 19:00:00'],
              dtype='datetime64[ns]', length=644059, freq=None)

## *ee.ImageCollections* exportieren

Jetzt können wir Daten aussuchen und exportieren:

In [25]:
roi = ee.Geometry.Rectangle([8.1, 48.9, 8.7, 49.1])

collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterBounds(roi) \
    .filterDate('2021-03-31T00:00:00', '2021-03-31T23:00:00') \
    .filter(ee.Filter.listContains("system:band_names", "temperature_2m"))

geemap.ee_export_image_collection(collection, region=roi, out_dir='data')

Total number of images: 23

Exporting 1/23: data\20210331T00.tif
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data\20210331T00.tif


Exporting 2/23: data\20210331T01.tif
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data\20210331T01.tif


Exporting 3/23: data\20210331T02.tif
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data\20210331T02.tif


Exporting 4/23: data\20210331T03.tif
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data\20210331T03.tif


Exporting 5/23: data\20210331T04.tif
Generating URL ...
Please wait ...
Data downloaded to D:\GitHub\Geodatenanalyse-II\Big Data (BD)\Termin-5_(Gabriel)\data\20210331T04.tif


Exporting 6/23: data\20210331T05.tif
Generating URL ...
Please wait ...
Data downloaded to D

## Zeitreihen in *Pandas* exportieren

Daten aus *ee.ImageCollections* können auch als Zeitreihe extrahiert werden:

In [30]:
KA = ee.Geometry.Point(8.405, 49.014)
buffer = 50

land_temp = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterDate('2021-01-01T00:00:00', '2021-03-31T00:00:00')

Schauen wir uns einen beispielhaften Datenpunkt an:

In [31]:
# Calculate and print the mean value of the LST collection at the point
land_temp_point = land_temp.mean().sample(KA, buffer).first().get('temperature_2m').getInfo()
print('Average for KA at 1 Jan 2021 00:00:', round(land_temp_point - 273.15, 2), '°C')

Average for KA at 1 Jan 2021 00:00: 4.08 °C


Laden wir die Zeitreihe für Karlsruhe herunter (das kann eine Weile dauern):

In [32]:
# Get the data for the point in urban area.
land_temp_poi = land_temp.getRegion(KA, buffer).getInfo()
len(land_temp_poi)

2137

Wir definieren eine Funktion zur Speicherung der Daten als *pandas.DataFrame*:

In [33]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

Nun können wir die gewünschten Daten extrahieren:

In [34]:
land_temp_df = ee_array_to_df(land_temp_poi, ['temperature_2m'])

# konvertiere in Celsius
land_temp_df['temperature_2m'] -= 273.15

# als CSV speichern
land_temp_df.to_csv("data/KA_temperature-2m.csv")

land_temp_df

,time,datetime,temperature_2m
0,1609459200000,2021-01-01 00:00:00,3.056772
1,1609462800000,2021-01-01 01:00:00,2.838892
2,1609466400000,2021-01-01 02:00:00,2.622156
3,1609470000000,2021-01-01 03:00:00,2.602029
4,1609473600000,2021-01-01 04:00:00,2.367548
...,...,...,...
2131,1617130800000,2021-03-30 19:00:00,14.312723
2132,1617134400000,2021-03-30 20:00:00,11.957925
2133,1617138000000,2021-03-30 21:00:00,10.556375
2134,1617141600000,2021-03-30 22:00:00,9.288660


## ENDE